In [ ]:
import time
import random 

import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup

import click

from whyclick.chrome import open_chrome, remove_popups
from whyclick import whyq

import torch

In [ ]:
query = {'username': "*****", 'password': "*****"}

driver = whyq.login(query['username'], query['password'], headless=False)

In [ ]:
order_json = whyq.download_previous_orders(driver)

In [ ]:
ordered_items = [o['Item Name'] for o in order_json]

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
def vectorize(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
    ff = model(input_ids)
    return ff[0].squeeze().squeeze().detach().numpy()[0]


In [ ]:
# Converts sentences to arrays of floats.
vectorized_sents = [vectorize(s) for s in tqdm(ordered_items)]

In [ ]:
driver.get('http://whyq.sg')

In [ ]:
def smartly_order_one_day(driver, element_day, halal=False, healthy=False, vegetarian=False):
    time.sleep(1)
    loop_count = 0 # Sanity break.
    while True:
        try:
            # Apply dietary filter
            whyq.apply_dietary_filters(driver, halal, healthy, vegetarian)
            # Find meals.
            meals = [b for b in element_day.find_elements_by_xpath('//button')
                     if b and b.text == "ADD"]

            meals_str = [h6.text for h6 in element_day.find_elements_by_xpath('//h6')][:len(meals)]
            meals_vec = [vectorize(m) for m in meals_str]

            top_m, top_m_score = -1, -1
            for i, m in enumerate(meals_vec):
                for s in vectorized_sents:
                    if np.dot(m, s.T) > top_m_score:
                        top_m = i

            meals[top_m].click()

            # Randomly choose one.
            ##random.choice(meals).click()

            # Check if you've ordered already.
            time.sleep(0.3)
            msg = element_day.find_element_by_xpath('//div[@id="notify_msg"]')
            break
        except IndexError: # No meals from dietary restriction.
            # Repeat the loop so that the filters are undone.
            pass
        if loop_count > 3: # Sanity break.
            break # If everything fails, go to next day.
        loop_count += 1
    return driver, element_day, msg.text

In [ ]:
days = driver.find_elements_by_xpath("//div[@class='owl-item active']")

for element_day in days:
    element_day.click()
    driver, element_day, msg = smartly_order_one_day(
        driver, element_day, halal=True
    )
    time.sleep(3)

In [ ]:
driver.find_element_by_link_text("PLACE ORDER").click()